In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [51]:
team_stats = pd.read_csv('../data/FS_champ_team_2021.csv')
games = pd.read_csv('../data/FS_champ_games_2021.csv')

In [52]:
team_stats.describe()

,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,wins_away,draws,draws_home,draws_away,...,goals_conceded_min_61_to_70,goals_conceded_min_71_to_80,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage
count,24.000000,24.000000,24.000000,24.0,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,46.416667,23.208333,23.208333,0.0,17.375000,9.583333,7.791667,11.666667,5.833333,5.833333,...,6.625000,5.541667,9.208333,25.166667,25.000000,25.125000,37.458333,33.708333,41.333333,4.583333
std,0.974308,0.508977,0.508977,0.0,5.427887,3.399702,2.918891,2.632021,2.119612,2.258639,...,2.533472,2.395270,3.833255,5.654292,8.968544,9.728097,11.754663,14.131307,14.187155,3.295803
min,46.000000,23.000000,23.000000,0.0,11.000000,5.000000,4.000000,6.000000,2.000000,2.000000,...,3.000000,1.000000,4.000000,13.000000,9.000000,9.000000,15.000000,8.000000,17.000000,0.000000
25%,46.000000,23.000000,23.000000,0.0,12.750000,7.000000,6.000000,10.000000,4.000000,4.000000,...,4.750000,4.000000,6.000000,22.000000,17.000000,17.000000,30.000000,26.000000,30.000000,2.000000
50%,46.000000,23.000000,23.000000,0.0,16.000000,8.500000,7.500000,11.000000,6.000000,6.000000,...,6.500000,5.000000,9.000000,24.000000,25.500000,25.500000,39.000000,32.500000,38.500000,4.000000
75%,46.000000,23.000000,23.000000,0.0,20.000000,12.000000,9.250000,13.000000,7.000000,8.000000,...,8.250000,7.250000,11.000000,28.000000,30.000000,33.500000,46.000000,40.000000,52.000000,6.250000
max,49.000000,25.000000,25.000000,0.0,29.000000,19.000000,15.000000,17.000000,10.000000,10.000000,...,12.000000,10.000000,17.000000,37.000000,43.000000,43.000000,57.000000,61.000000,70.000000,13.000000


In [53]:
games.describe()

,timestamp,attendance,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,...,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no
count,5.570000e+02,0.0,552.00000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,...,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000,557.000000
mean,1.610614e+09,NaN,23.50000,1.402819,1.216391,1.489587,1.258833,1.253142,1.052065,2.305206,...,3.004165,2.613393,3.388546,3.425189,1.390557,2.141490,3.845907,7.589318,1.932047,1.793914
std,5.939879e+06,NaN,13.28796,0.675394,0.604858,0.398057,0.353639,1.183905,1.013834,1.491049,...,0.934704,0.862611,0.369592,1.333672,0.083534,0.231239,0.609371,1.478715,0.142656,0.128460
min,1.599850e+09,NaN,1.00000,0.000000,0.000000,0.830000,0.650000,0.000000,0.000000,0.000000,...,0.000000,1.250000,2.900000,1.380000,1.150000,1.500000,2.300000,3.900000,1.540000,1.540000
25%,1.605971e+09,NaN,12.00000,1.000000,0.830000,1.130000,1.000000,0.000000,0.000000,1.000000,...,2.710000,2.000000,3.200000,2.540000,1.330000,2.000000,3.400000,6.700000,1.830000,1.710000
50%,1.610809e+09,NaN,23.50000,1.440000,1.220000,1.430000,1.300000,1.000000,1.000000,2.000000,...,3.140000,2.450000,3.300000,3.100000,1.380000,2.110000,3.800000,7.500000,1.950000,1.770000
75%,1.615638e+09,NaN,35.00000,1.830000,1.570000,1.750000,1.520000,2.000000,2.000000,3.000000,...,3.510000,3.000000,3.500000,4.000000,1.450000,2.300000,4.250000,8.500000,2.000000,1.870000
max,1.622297e+09,NaN,46.00000,3.000000,3.000000,2.570000,2.130000,7.000000,5.000000,9.000000,...,7.000000,8.500000,6.500000,12.000000,1.620000,2.850000,6.500000,16.000000,2.350000,2.300000


# Goal Analysis

In [69]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [65]:
print(f'There were {season_goal_total} goals scored this season')

There were 1284 goals scored this season


In [66]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 575 goals scored in the 1H this season, 


In [68]:
print(f'')

# In how many games did a team have more xG but lose?

In [41]:
games['morexg_but_loss'] = np.where(np.logical_and(np.greater(games['away_team_goal_count'], games['home_team_goal_count']),
                                    games['team_a_xg'],games['team_b_xg'],
                                    1, 0))

TypeError: ufunc() takes from 2 to 3arguments but 5 were given